<a href="https://colab.research.google.com/github/Puneet-Bajaj-IITM/01245/blob/main/Chat_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!curl https://ollama.ai/install.sh | sh
!nohup ollama serve &
!pip install ollama
!ollama pull llama2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10044    0 10044    0     0  39698      0 --:--:-- --:--:-- --:--:-- 39857
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


nohup: appending output to 'nohup.out'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00


pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling ma

In [18]:
import ollama

In [19]:
res = ollama.generate(model='llama2', prompt=prompt)

In [21]:
print(res['response'])


Summary of Chat:

The customer, Karen, contacted Adobe support to change their plan from 4 licenses to 2 licenses due to semi-retirement. The support agent assisted Karen in cancelling the extra two licenses and provided a confirmation number for the cancellation. The agent also offered a discount on one of the remaining licenses if Karen wished to keep all four licenses.

Issues Encountered:

* Customer desired to reduce the number of licenses from 4 to 2 due to semi-retirement.
* Agent assisted customer in cancelling the extra two licenses.

Solutions Provided:

* Cancellation of two licenses.
* Offer for a discount on one remaining license if customer wished to keep all four licenses.

General Themes Explored:

* Customer retirement and reduced license needs.
* Agent assistance in cancelling licenses and providing discounts.

Recurring Issues:

* Customers downsizing or upgrading their plans due to changes in their needs.
* Agents assisting customers in cancelling or modifying thei

In [22]:
import pandas as pd

In [23]:
data = pd.read_excel('/content/file.xlsx', header=None)

In [26]:
data.head()

,0,Summary
0,[customer connecting with agent - disregard] I...,None
1,[customer connecting with agent - disregard] w...,None
2,[customer connecting with agent - disregard] h...,None
3,[customer connecting with agent - disregard] W...,None
4,[customer connecting with agent - disregard] I...,None


In [25]:
data['Summary'] = None
data.columns = ['Chat', 'Summary']

In [103]:
for index, row in data.iterrows():
  if row['Chat'] is not None:
    res = ollama.generate(model='llama2', prompt=f"""Summarize {row['Chat']}, mention the main issues encountered by the customer, the solutions provided by agent and overarching themes discussed""")
    # Assuming 'Summary' is a new column to store generated summaries
    data.at[index, 'Summary'] = res['response']



In [104]:
data.head()

,Chat,Summary
0,[customer connecting with agent - disregard] I...,The conversation between Karen and the Adobe V...
1,[customer connecting with agent - disregard] w...,The conversation is between an Adobe Virtual A...
2,[customer connecting with agent - disregard] h...,"The customer, Ann, was experiencing issues wit..."
3,[customer connecting with agent - disregard] W...,The customer is experiencing issues with Adobe...
4,[customer connecting with agent - disregard] I...,The conversation between the customer and the ...


In [38]:
import concurrent.futures

def generate_summary(row):
    res = ollama.generate(model='llama2', prompt=f"""Summarize {row['Chat']}, mention the main issues encountered by the customer, the solutions provided by agent and overarching themes discussed""")
    return res['response']

# Define a function to process each row asynchronously
def process_row(row):
    summary = generate_summary(row)
    return summary

# Create a ThreadPoolExecutor to execute the processing function asynchronously
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit processing tasks for each row in the DataFrame
    futures = [executor.submit(process_row, row) for _, row in data.iterrows()]

    # Retrieve the results of the asynchronous tasks
    for future, (index, _) in zip(futures, data.iterrows()):
        summary = future.result()
        data.at[index, 'Summary'] = summary


In [40]:
print(data['Summary'][10])


Chat Summary:

The customer encountered several issues with their account, including difficulty logging in, forgotten password, and issues with their payment method. The support agent provided step-by-step instructions on how to reset the password, assisted the customer in updating their payment information, and ensured that the customer was able to log in successfully.

Main Issues Encountered:

1. Difficulty logging in due to forgotten password or other technical issues.
2. Issues with payment methods, including expired cards or failed transactions.

Solutions Provided:

1. Resetting the password through a simple process involving email verification.
2. Updating payment information and ensuring that it is current and valid.
3. Troubleshooting technical issues to ensure successful login.

Overarching Themes Discussed:

1. The importance of keeping personal information up to date, including payment methods, to avoid inconvenience or difficulty in accessing services.
2. The need for a 

KeyError: 'passes columns are not ALL present dataframe'

In [105]:
data.to_excel("output.xlsx", index=False, columns=['Chat', 'Summary'])

In [106]:
data.tail(1)

,Chat,Summary
59,"Due to the large size of your report, exports ...",\nBased on the information provided in the rep...
